In [20]:
import requests
from bs4 import BeautifulSoup
from lxml import etree as et
import time
import csv

In [3]:
url = "https://www.amazon.com/Apple-MacBook-Laptop-12%E2%80%91core-19%E2%80%91core/dp/B0BSHDJG9T/ref=pd_ci_mcx_mh_mcx_views_0?pd_rd_w=wm3h2&content-id=amzn1.sym.8b590b55-908d-4829-9f90-4c8752768e8b%3Aamzn1.symc.40e6a10e-cbc4-4fa5-81e3-4435ff64d03b&pf_rd_p=8b590b55-908d-4829-9f90-4c8752768e8b&pf_rd_r=Z8ECMH9ABJ02EA0R5K46&pd_rd_wg=0SYTs&pd_rd_r=2b721d28-423e-4783-8caa-098bb631bd20&pd_rd_i=B0BSHDJG9T"


In [22]:
#first method tried, essentially detected and blocked by Amazon
def get_amazon_price(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Try to find the price element by class
    price_element = soup.find('span', class_="a-price-whole")
    return price_element
    if price_element:
        try:
            # Retrieve the price text and clean it up
            price_text = price_element.text.strip().replace(',', '').replace('.', '')
            return float(price_text)
        except ValueError:
            return "Error converting price to number"

    return "Price unavailable"

In [32]:
#I looked at the output in this and realized that Amazon was redirecting me to a captcha instead of 
#the item page because it suspected a bot, so I switched to using selenium instead
response = requests.get(link, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
print(soup)

<!DOCTYPE html>

<!--[if lt IE 7]> <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8 a-lt-ie7"> <![endif]-->
<!--[if IE 7]>    <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8"> <![endif]-->
<!--[if IE 8]>    <html lang="en-us" class="a-no-js a-lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="a-no-js" lang="en-us"><!--<![endif]--><head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title dir="ltr">Amazon.com</title>
<meta content="width=device-width" name="viewport"/>
<link href="https://images-na.ssl-images-amazon.com/images/G/01/AUIClients/AmazonUI-3c913031596ca78a3768f4e934b1cc02ce238101.secure.min._V1_.css" rel="stylesheet"/>
<script>

if (true === true) {
    var ue_t0 = (+ new Date()),
        ue_csm = window,
        ue = { t0: ue_t0, d: function() { return (+new Date() - ue_t0); } },
        ue_furl = "fls-na.amazon.com",
        ue_mid = "ATVPDKIKX0DER"

In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# Set Chrome optionsm to run headless
chrome_options = Options()
chrome_options.add_argument("--headless") 

def return_price(url_list):
    # Initialize WebDriver using ChromeDriver Manager
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)  
    return_list = []
    
    for url in url_list:
        driver.get(url)
        
        # Extract the HTML page source
        page_content = driver.page_source
        
        # use BeautifulSoup for parsing
        soup = BeautifulSoup(page_content, 'html.parser')
        
        # Print parsed content for debugging purposes
        # print(soup.prettify())
    
        #find the name and price of the item by the class/id elements
        price_int = soup.find('span', class_="a-price-whole")
        price_decimal = soup.find('span', class_='a-price-fraction')
        price_total = price_int.get_text() + price_decimal.get_text()
        product_name = soup.find(id = 'productTitle').get_text().strip()

        return_list.append((product_name, price_total))
        
    
    # Close the browser
    driver.quit()
    
    return return_list

In [30]:
item_list = ['https://www.amazon.com/Apple-MacBook-Laptop-12%E2%80%91core-19%E2%80%91core/dp/B0BSHDJG9T/ref=pd_ci_mcx_mh_mcx_views_0?pd_rd_w=wm3h2&content-id=amzn1.sym.8b590b55-908d-4829-9f90-4c8752768e8b%3Aamzn1.symc.40e6a10e-cbc4-4fa5-81e3-4435ff64d03b&pf_rd_p=8b590b55-908d-4829-9f90-4c8752768e8b&pf_rd_r=Z8ECMH9ABJ02EA0R5K46&pd_rd_wg=0SYTs&pd_rd_r=2b721d28-423e-4783-8caa-098bb631bd20&pd_rd_i=B0BSHDJG9T', 'https://www.amazon.com/Harvard-16-oz-Ceramic-Mug/dp/B0B2ZB4XMZ/?_encoding=UTF8&pd_rd_w=LHfmo&content-id=amzn1.sym.3c3990c3-513c-4686-8d92-a42b4095cecb%3Aamzn1.symc.8b620bc3-61d8-46b3-abd9-110539785634&pf_rd_p=3c3990c3-513c-4686-8d92-a42b4095cecb&pf_rd_r=0RZA1NMRJ8SH5MJTYVQM&pd_rd_wg=RMfzY&pd_rd_r=23ac4591-73f2-48fa-b640-a781197701ba&ref_=pd_hp_d_btf_ci_mcx_mr_hp_d&th=1']
return_price(item_list)

[('Apple 2023 MacBook Pro Laptop M2 Pro chip with 12‑core CPU and 19‑core GPU: 16.2-inch Liquid Retina XDR Display, 16GB Unified Memory, 512GB SSD Storage. Works with iPhone/iPad; Space Gray',
  '1,998.99'),
 ('RFSJ 16 Oz Ceramic Mug (Harvard)', '24.99')]

In [1]:
import yfinance as yf

# Define stock tickers
stock_tickers = ['AAPL', 'GOOGL', 'TSLA']\

# Define function to get the current stock price
def fetch_current_price(ticker):
    stock = yf.Ticker(ticker)
    current_price = stock.history(period='1d')['Close'].iloc[0]
    return current_price

# Fetch and print current prices for each ticker
for ticker in stock_tickers:
    price = fetch_current_price(ticker)
    print(f"Current price of {ticker}: ${price:.2f}")

Current price of AAPL: $182.40
Current price of GOOGL: $171.25
Current price of TSLA: $177.81
